In [8]:
import numpy as np
import trimesh as tm
import matplotlib.pyplot as plt
from scipy.spatial import Delaunay as dl
import random 

random.seed(1)

In [14]:
def my_triangulation(points):
    return (dl(points[:,(0,1)]).simplices) 

def my_mesh_builder(points, triangles):
    return tm.Trimesh(vertices=points, faces=triangles)
|
class Individual:

    def __init__(self, original_number_of_points, original_points, original_area, keep_or_leave = None):

        self.keep_or_leave = keep_or_leave

        if(keep_or_leave == None):
            self.generate_keep_or_leave(original_number_of_points)
        
        self.number_of_points = None
        self.generate_number_of_points()

        self.points = []
        self.generate_points(original_number_of_points, original_points)

        self.area = my_mesh_builder(self.points, my_triangulation(self.points)).area

        self.error = np.abs(original_area - self.area)

        self.distance = None #!!

    def generate_keep_or_leave(self, original_number_of_points):
        self.keep_or_leave = [random.randint(0,1) for _ in range(original_number_of_points)]

    def generate_number_of_points(self):
        self.number_of_points = sum(self.keep_or_leave)

        while(self.number_of_points <= 2):
            i = 0
            if(self.keep_or_leave[i] == 0):
                self.keep_or_leave[i] = 1
                self.number_of_points = self.number_of_points + 1
            i = i + 1

    def generate_points(self, original_number_of_points, original_points):
        for i in range(original_number_of_points):
            if(self.keep_or_leave[i] == 1):
                self.points.append(original_points[i])
        self.points = np.array(self.points)
    
    def __str__(self):
        show_k_or_l = "Keep or leave: \n" + str(self.keep_or_leave) + "\n"
        show_number_of_points = "Number of points: \n" + str(self.number_of_points) + "\n"
        show_points = "Points: \n" + str(self.points) + "\n"
        show_area = "Area: \n" + str(self.area) + "\n"
        show_error = "Error: \n" + str(self.error) + "\n"
        show_distance = "Distance: \n" + str(self.distance) + "\n"

        return show_k_or_l + show_number_of_points + show_points + show_area + show_error + show_distance

class My_Mesh_Simp:

    def __init__(self,original_mesh, population_size):

        self.original_points = original_mesh.vertices
        self.original_number_of_points = len(self.original_points)
        self.original_area = original_mesh.area

        self.population_size = population_size
        self.population = [] #!!
        self.generate_first_population(self.original_number_of_points, self.original_points, self.original_area)

        self.front1 = None #!!
        self.front2 = None #!!
        self.front3 = None #!!
    
    def generate_first_population(self, original_number_of_points, original_points, original_area):
        for _ in range(self.population_size):
            self.population.append(Individual(original_number_of_points, original_points, original_area))

    def calculate_fronts(self):
        pass

    def calculate_distances(self):
        pass

    def select_the_fittest(self):
        pass

    def generate_offspring(self):
        pass





        


In [10]:
ind1 = Individual(5,[[1,0,3],[4,5,6],[7,8,9],[10,11,12],[13,14,15]], 12)
print(ind1)

Keep or leave: 
[1, 0, 1, 0, 1]
Number of points: 
3
Points: 
[[ 1  0  3]
 [ 7  8  9]
 [13 14 15]]
Area: 
8.48528137423857
Error: 
3.5147186257614305
Distance: 
None

